# Homework 2 
## Searching algorithms for optimal decision-making in game theory and AI

### Task 1

Given a larger board, where width= 7 and height= 6, define the class ConnectFour which is a version of tic-tac-toe implemented before. 
Hint: class ConnectFour inherits class TicTacToe and has an additional function which constraints the player to operate only in the lowest empty square of a column. 
For further references Play Connect Four online and read more details here.

In [103]:
#In order to create the class ConnectFour, I'm going to need most of the classes from Lab 6.

from collections import defaultdict
import random
import math
import functools 
import copy

# The first needed class is Board (adapted to the size of a Connect Four Board)
# Class Board is a mathematical representation of the problem, holding the legal states and possible moves for the 
# allowed positions. Board overrides clas defaultdict. Method __missing__ from defaultdict is used to manage empty
# fields or to assingn corresponding values for each position. 
# Structure of the Board: {(coordinates as tuple) : attributes}

class Board(defaultdict):
    empty = '.'
    used = '#'
    
    # For Connect Four, the Board must have a width of 7 cells and a height of 6
    def __init__(self, width=7, height=6, current_player=None, **kwds):
        self.__dict__.update(width=width, height=height, current_player=current_player, **kwds)
    
    # Given the position of a cell, verify if its coordinates are within the boundaries of the board and mark the cell 
    # as an empty square. Otherwise, the cell will be marked as `used`.
    def __missing__(self, pos):
        (x,y) = pos
        if (x,y) in self.__dict__.keys():
            return self.__dict__[(x,y)]
        if x>=0 and x<self.width and y>=0 and y<self.height:
            return self.empty
        else:
            return self.used
    
    # Hash method stores the instances in hash tables
    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.current_player)
    
    def __repr__(self):
        def row(y): 
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'
    
    # Update the Board with the new changes of each cell
    def update_board(self, changes:dict, **kwds)->'Board':
        self.__dict__.update(self)
        self.__dict__.update(changes)
        return board
    
board = Board(width=7,height=6,current_player='R')
board.update_board({(0,0):'R', (0,1):'R', (0,2):'R', (0,3):'R'})
print(board)

R . . . . . .
R . . . . . .
R . . . . . .
R . . . . . .
. . . . . . .
. . . . . . .



In [192]:
# Next, I'll ned a helper function to count if the player has similar symbols in a line.The player variable represents 
# the player we are counting for (red or blue in this case). The square variable can be used to only check lines that 
# contain a specific position.
def k_in_row(board, player, square, k):
    (x,y) = square
    no = 0
    row = row_check(board,square,k)
    rnum = count(row,player)
    if rnum>no:
        no = rnum
    col = col_check(board,square,k)
    cnum = count(col,player)
    if cnum>no:
        no = cnum
    if no>=k:
        return True
    else:
        return False

def row_check(board, square, k):
    (x,y) = square
    row = []
    for i in range(board.__dict__['width']):
        if (i,y) in board.__dict__:
            val = board.__dict__[(i,y)]
            row.append(val)
        else:
            row.append('.')
    #print(row)
    return row
    
def col_check(board, square, k):
    (x,y) = square
    col = []
    for i in range(board.__dict__['height']):
        if (x,i) in board.__dict__:
            val = board.__dict__[(x,i)]
            col.append(val)
        else:
            col.append('.')
    #print(col)
    return col

def count(l, player):
    num = 0
    for i in l:
        if i==player:
            num += 1
            if num==4:
                return num
        else:
            num=0
    return num

k_in_row(board, 'R', (0,0), 4)

True

In [195]:
# Class ConnectFour should extend class TicTacToe, but there were so many errors in my old code I decided to discard it
class ConnectFour():
    # A list of players:
    players = ['R', 'B']
    
    #Firstly, different width, height and required similar symbols in a row
    def __init__(self, width=7, height=6, k=4):
        self.k = k
        self.width = 7
        self.height = 6
        # The set of all the points on the board
        self.squares = []
        for x in range(self.width):
            for y in range(self.height):
                self.squares.append((x,y))
        # The board where Red plays first, and the utility value is 0.
        self.initial = Board(height=self.height, width=self.width, current_player='R', utility=0)
        
    # Next, a player can only occupy a cell if it's the lowest unoccupied cell on it's column
    def actions(self, board):
        allowed = []
        # Check all the squares in the ConnectFour class to see if they are empty on the Board  and it is possible to
        # occupy them
        for (x,y) in self.squares:
            if self.is_possible(board, (x,y)):
                allowed.append((x,y))
        return allowed
    
    # Function which allows a cell to be occupied only if it is the lowest unoccupied cell on its column: if they have 
    # the y coordinate equal to k(5) (they are at the bottom of the board) or the cell beneath (with y+1) is occupied 
    # (is in the dictionary of the Board)
    def is_possible(self, board, square):
        (x,y)=square
        if (x,y) not in board.__dict__.keys():
                if y==self.height-1 or (x,y+1) in board.__dict__.keys():
                    return True
                else:
                    return False
    
    # Now let's figure out the utility of a certain board for a certain player
    def utility(self, board, player):
        # I will use the k_in_row function. If it returns True for this same player, utility is 1, if it returns True
        # for the other player, utility is -1. Else, utility is 0.
        for p in self.players:
            # k_in_row requiares a specific square, so we check all of them. Break if returns True.
            for square in self.squares:
                if k_in_row(board,p,square,self.k):
                    if p==player:
                        #if the player for which k_in_row returns True coincides iwth the one we're checking for, they won
                        return 1
                    else:
                        #if k_in_row returns True for a different player, the player we're checking for has lost
                        return -1
        #In all other situations, we have a tie
        return 0
    
    def make_move(self, board, square):
        if not self.end(board):
            # Occupy the cell with the current player's symbol
            board.update_board({square:board.__dict__['current_player']})
            # Select the next player in the list to become the current player

            if board.__dict__['current_player']=='R':
                board.__dict__.update({'current_player':'B'})
            else:
                board.__dict__.update({'current_player':'R'})
        return board
        
    #Let's check if end conditions are fulfilled
    def end(self, board):
        numpos = len(self.actions(board))
        val = self.utility(board, board.current_player)
        if numpos == 0 or val!=0:
            return True
        return False

cf = ConnectFour(width=7, height=6, k=4)
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
cf.make_move(cf.initial,(0,5))
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
cf.make_move(cf.initial,(1,5))
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
cf.make_move(cf.initial,(0,4))
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
cf.make_move(cf.initial,(1,4))
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
cf.make_move(cf.initial,(0,3))
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
cf.make_move(cf.initial,(1,3))
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
cf.make_move(cf.initial,(0,2))
print(cf.initial)
print('Allowed actions:', cf.actions(cf.initial))
print('Utility:', cf.utility(cf.initial,'R'))
print('End: ',cf.end(cf.initial))

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Allowed actions: [(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5)]
Utility: 0
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
R . . . . . .

Allowed actions: [(0, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5)]
Utility: 0
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
R B . . . . .

Allowed actions: [(0, 4), (1, 4), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5)]
Utility: 0
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
R . . . . . .
R B . . . . .

Allowed actions: [(0, 3), (1, 4), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5)]
Utility: 0
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
R B . . . . .
R B . . . . .

Allowed actions: [(0, 3), (1, 3), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5)]
Utility: 0
. . . . . . .
. . . . . . .
. . . . . . .
R . . . . . .
R B . . . . .
R B . . . . .

Allowed actions: [(0, 2), (1, 3), (2, 5), (3, 5), (4, 5), (5, 5),

### Task 2

Implement the algorithm Upper Confidence Bound for Trees from the Monte Carlo Tree Search Algorithm family.

#### Representing a node in MCTS as a class:
• the number of visits, N, representing the number of simulations performed from a node or a descendant of it.
• the estimated value, Q, indicating the quality of the node,  based on the number of games won starting from that node.
• Nnode, a reference to the parent node.
• children, a dictionary that contains for each action a link to the next node.
• c is a constant in the exploration and can be adjusted and can have a default value of 1√2

In [196]:
class Node():
    def __init__(self, board, N, Q, N_node, children:dict):
        self.board = board
        self.N = N
        self.Q = Q
        self.N_node = N_node
        self.children = children

In [213]:
def random_player(game, board):
    board = game.initial
    print(board)
    actions = game.actions(board)
    #print(game.actions(board))
    chosen = random.choice(actions)
    return game.make_move(board,chosen)

g = ConnectFour()
while not g.end(g.initial):
    g.initial = random_player(g, g.initial)
    print(g.initial)

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . R . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . R . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . B
. . R . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . B
. . R . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . R . . . B
. . R . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . R . . . B
. . R . R . B

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . R . .

In [215]:
# Function that returns the next board considered best by an MCTS algorithm
def mcts_player(game, board):
    C_p = 1/math.sqrt(2)
    
    def tree_policy(v):
        while not game.end(v.board):
            actions = game.actions(v.board)
            for a in actions:
                if a not in v.children.keys():
                    return expand(v)
                else:
                    v = best_child(v,C_p)
        return v
        
    def expand(v):
        actions = game.actions(v.board)
        for a in actions:
            if a not in v.children:
                b = copy.deepcopy(v.board)
                v.children[a] = Node(game.make_move(b, a),1,0,v,{})
        print(v.children[a])
        return v.children[a]
                
    def best_child(v,c):
        max = -math.inf
        max_move = None
        index = False
        print(v.children)
        for key in v.children.keys():
            print(v.children[key])
            val = v.children[key]
            m = val.Q/val.N + c*math.sqrt(2*math.log(v.N)/val.N)
            if m>=max:
                index = True
                max = m
                max_move = key
                max_child = val
        return max_child
    
    def default_policy(board):
        player = board.__dict__['current_player']
        while not game.end(board):
            move = random_player(game, board)
            game.make_move(board, move)
        return game.utility(board, player)
    
    def backup(v, delta):
        while v!=None:
            v.N += 1
            v.Q += delta
            v = v.N_node
            
    # Create the root node v0 with the current board of the game and its current player, N=0, Q=0, no children found yet
    
    player = game.initial.__dict__['current_player']
    v0 = Node(board, N=0, Q=0, N_node=None, children={})
    #Let's say we have a computational budget of 10 iteratins
    for i in range(10):
        v1 = tree_policy(v0)
        delta = default_policy(v1.board)
        backup(v1, delta)
    bc = best_child(v0,0)
    return bc.board

In [216]:
def play_game(game, player_startegies:dict):
    while not game.end(game.initial):
        # R always starts
        game.initial = player_strategies['R'](game,game.initial)
        print(game.initial)
        # if R's move doesn't end the game, B makes their move in the same turn
        if not game.end(game.initial):
            game.initial = player_strategies['B'](game,game.initial)
            print(game.initial)
        else:
            break

In [217]:
# Test
player_strategies = {'R': mcts_player, 'B':random_player}
test_game = ConnectFour()
print(test_game.end(test_game.initial))
play_game(test_game,player_strategies)

#I'm not sure why this never stops running :(

False
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . R .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . B . . R .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . B . . R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B .
. . B . . R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B .
R . B . . R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B .
. . . . . B .
R . B . . R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B .
. . . . . B R
R . B . . R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B B
. . . . . B R
R . B . . R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B B
. . . . . B R
R . B . R R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B B
. . . . . B R
R . B B R R R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . B B
. .


. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R 


. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R 


. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R 


. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R 

...


. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R 


. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R 

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R


. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R R B R
R R B B R R R

. . . . . . .
. . . . . . .
. . . . . B .
. B B . . B B
B R R R 

KeyboardInterrupt: 

### Task 3

Define the query player(game, state)function to make the game more interactive and display the available/legalmoves.

In [218]:
def query_player(game, board):
    print('Possible moves:')
    actions = game.actions(board)
    for i in range(len(actions)):
        print('(', i, ') ', actions[i])
    move = input('Please, enter the index of your move:')
    return game.make_move(board, move)

In [222]:
#Test - Play against an MCTS algorithm
player_strategies2 = {'R': mcts_player, 'B':query_player}
test_game2 = ConnectFour()
play_game(test_game2,player_strategies)

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
B . . . R . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
B . . . R . R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
B . B . R . R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .
B . B . R . R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .
B . B . R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .
. . . . R . .
B . B . R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .
. . . . R . .
B . B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .
R . . . R . .
B . B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . . R . .
R . . B R . .
B . B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R


. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B 


. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B 


. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B 

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R


. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B 


. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B 


. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B 


. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B R . .
B B B B R B R

. . . . . . .
. . . . . . .
. . . . . . .
. . . R R . .
R . . B 

KeyboardInterrupt: 

# Feedback

I found ths homework really hard for a couple of reasons:

1. It was based on Lab 6, which I couldn't get to run properly at the time. I don't think this is very fair. Because I didn't manage to solve that lab, I had to reimplement everything in it, which is ok, but what if I still hadn't been able to? I wish you would have provided us with the solution for that lab after we submitted it so no one would be at a disadvantage when trying to solve this homework. I actually think that providing students with the correct solution of the lab after the deadline is a good practice. I was mostly unable to deliver fully complete and correct labs until now and I still don't know any better because I never got to see the correct solutions. I would actually love to see the solution for this homework as well.

2. All the tasks in this homework rely heavily on the ones before. For example, I couldn't figure out why task 2 has a never-ending output and now, task 3 has the same problem because it uses the same functions. This causes the risk that some students who couldn't do just task 1, might not be able to solve the whole thing and lose all the points.

I know I've been writing this at the end of every feedback lately :)), but I thought this was too hard for me. I'm not sure what level of dificulty you were aiming for, but I would consider this too hard for a homework.

Anyways, I look forward to seeing you again in class! Hope you're well :)

Irina Epure